<a href="https://colab.research.google.com/github/Mansi-Nayak/Tools_Langchain/blob/main/Tools_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script demonstrates how to create, invoke, and organize custom tools using LangChain's tooling system.
It starts by installing necessary dependencies including LangChain modules and DuckDuckGo search tools.
Then, it invokes a web search tool (`DuckDuckGoSearchRun`) to fetch the top news in India and displays the output.
It also uses the `ShellTool` to run basic shell commands (like `ls`) from Python in a controlled environment.
The script defines a basic `multiply` function and gradually converts it into a LangChain-compatible tool:
  - First by adding type hints,
  - Then by decorating it using `@tool`,
  - Then wrapping it into a `StructuredTool` with a Pydantic input schema,
  - Finally building a custom tool class by extending `BaseTool`.
Each version of the tool is invoked and its output, name, and descriptions are printed to show how tools are represented.
Additionally, the script creates multiple math tools (`add` and `multiply`) and bundles them inside a custom toolkit class.
This is useful in agent-based systems where tools can be exposed to LLM agents for interaction.
Overall, the code showcases how LangChain supports structured, schema-driven, reusable tool creation in Python.
"""

In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00


# **Built-in Tool - DuckDuckGo Search**

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

Get updated with current top news stories from India and the world only on Zee News. Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusive news stories on Indian politics, current affairs, cricket matches, festivals and ... The last rites of Irfan Shaikh (22), a crew member of the Air India flight AI 171 that crashed in Ahmedabad, were performed on Saturday in Nehru Nagar, Pimpri Chinchwad, Maharashtra. India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Top News Today. 0 / 0. The Hindu. About ... News India News Hyderabad news Bengaluru News Sports, IPL 2025 News Delhi news Live news Entertainment news Sitemap Trending on Group sites. Stock ... The tightrope: India walks a diplomatic tightrope as it has ties with both Ir

In [ ]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


# **Built-in Tool - Shell Tool**

In [ ]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# **Custom Tools**

In [ ]:
from langchain_core.tools import tool

In [ ]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [ ]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [ ]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [ ]:
result = multiply.invoke({"a":3, "b":5})

In [ ]:
print(result)

15


In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [ ]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# **Method 2 - Using StructuredTool**

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [ ]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [ ]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# **Method 3 - Using BaseTool Class**

In [ ]:
from langchain.tools import BaseTool
from typing import Type

In [ ]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [ ]:
multiply_tool = MultiplyTool()

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# **Toolkit**

In [ ]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [ ]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [ ]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
